In [1]:
!pip install ollama rdflib networkx faiss-cpu pandas numpy rapidfuzz pyshacl

  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached anyio-4.10.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp313-cp313-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached networkx-3.5-py3-none-any.whl (2.0 MB)
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ----------------------------- ---------- 11.0/14.9 MB 56.2 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#Cell 1 — Setup
# If needed, uncomment:
# !pip install ollama rdflib networkx faiss-cpu pandas numpy rapidfuzz pyshacl

import os, re, json, time, uuid, glob
from pathlib import Path
from typing import List, Dict, Any
import numpy as np
import pandas as pd
import ollama
import rdflib
from rdflib import Graph
from rdflib.namespace import RDF, RDFS, OWL, SKOS, XSD
import networkx as nx
import faiss
from rapidfuzz import fuzz

def cosine_sim(a: np.ndarray, b: np.ndarray) -> float:
    na = np.linalg.norm(a) + 1e-9
    nb = np.linalg.norm(b) + 1e-9
    return float((a @ b) / (na * nb))




In [ ]:
# Cell 2 — Config
DATA_DIR = Path("data")
ONTOLOGY_TTL = DATA_DIR / "upper_ontology.ttl"
CORPUS_DIR = DATA_DIR / "corpus"

LLM_MODEL   = os.environ.get("OLLAMA_LLM", "llama3.1:8b-instruct")
EMBED_MODEL = os.environ.get("OLLAMA_EMBED", "nomic-embed-text")

OUT_DIR = Path("outputs"); OUT_DIR.mkdir(parents=True, exist_ok=True)

# Retrieval / alignment knobs
K_ENTITY = 60
K_HYPEREDGE = 60
ENTITY_ALIGN_MIN = 0.55
NEW_CONCEPT_FREQ_MIN = 2

DATA_DIR.mkdir(exist_ok=True); CORPUS_DIR.mkdir(exist_ok=True)

print("LLM:", LLM_MODEL, "| EMBED:", EMBED_MODEL)
print("Data:", DATA_DIR.resolve(), "| Output:", OUT_DIR.resolve())
